# Introduction to LangChain v0.1.0 and LCEL: LangChain Powered RAG

## Videos to Watch:

[Langchain](https://www.youtube.com/watch?v=Azfc-TjG9Tg)

## Start of Hands-on:

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

In the notebook, you'll complete the following Tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Initialize a Simple Chain using LCEL
  4. Implement Naive RAG using LCEL

- 🤝 Breakout Room #2:
  1. Create a Simple RAG Application Using QDrant, OpenAI, and LCEL

Let's get started!



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [1]:
%pip install -qU langchain langchain-core langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


Now we can get our Qdrant dependencies!

In [2]:
%pip install -qU qdrant-client

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.17.0 requires protobuf!=4.21.0,<5,>=3.19.0; sys_platform != "linux", but you have protobuf 5.26.1 which is incompatible.


Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [3]:
%pip install -qU tiktoken pymupdf

Note: you may need to restart the kernel to use updated packages.


## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-3.5-turbo` model.

In [5]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

####❓ Question #1:

What specific model are we using when we point to `gpt-3.5-turbo`?

> HINT: Check out [this page](https://platform.openai.com/docs/models/gpt-3-5-turbo) to find the answer!

- gpt-3.5-turbo-0125

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a Senior Software Engineer with 30+ years work experience in using NextJs and React."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/docs/expression_language/), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [7]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [8]:
print(chain.invoke({"content": "Hello world in TypeScript!"}))

content='```typescript\nconsole.log("Hello world!");\n```' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-75d24f1e-6cab-49e6-9959-1b82736206b7-0'


Let's try it out with a different prompt!

In [9]:
chain.invoke({"content" : "Could I please have some advice on how to become a better NextJS Programmer?"})

AIMessage(content="Absolutely! Here are some tips to help you become a better Next.js programmer:\n\n1. **Understand the Basics**: Make sure you have a strong understanding of React and its core concepts before diving into Next.js. Understanding how React works will help you better utilize Next.js features.\n\n2. **Learn Next.js Concepts**: Familiarize yourself with key Next.js concepts such as server-side rendering, static site generation, dynamic routing, and data fetching. Understanding these concepts will help you leverage Next.js to its full potential.\n\n3. **Practice, Practice, Practice**: The best way to improve your Next.js skills is through practice. Build projects, experiment with different features, and try to solve real-world problems using Next.js.\n\n4. **Stay Updated**: Next.js is constantly evolving, with new features and updates being released regularly. Stay updated with the latest Next.js documentation, blog posts, and community discussions to keep up with the lates

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually Adding Context

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [10]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content='LangChain is a blockchain platform that focuses on multilingual translation and interpretation services. It aims to connect users with language experts around the world using blockchain technology to ensure secure and efficient transactions. LangChain can help businesses and individuals overcome language barriers and facilitate communication in a globalized world.' response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d47249ca-66af-4f21-8275-53c2d716c0c8-0'


Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [11]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

content='LangChain Expression Language (LECL) is a domain-specific language designed for specifying and manipulating language data in the LangChain platform. LECL allows users to define patterns, transformations, and rules for processing and transforming language data within the LangChain environment. It is used to create language models, extract information from text, and perform various natural language processing tasks.' response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 27, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-cee3919d-3601-495e-8517-c440ad2df460-0'


While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [12]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language or LCEL is a declarative way to easily compose chains together. It provides benefits such as async, batch, and streaming support, fallback handling, parallelism, and seamless integration with LangSmith Tracing for observability and debuggability of complex chains.' response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 274, 'total_tokens': 330}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7075d519-af7a-4248-8351-ef151d2c3abd-0'


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #2: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

In [13]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [14]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [19]:
print(len(enc.encode(context)))
print(enc.encode(context))
print(context)

11
[198, 17211, 56, 473, 26043, 24860, 80942, 13575, 95757, 48198, 198]

EVERY HITCHHIKER'S GUIDE BOOK



The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-32k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [75]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [69]:
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(text)
    return len(tokens)

class EnhancedTextSplitter:
    def __init__(self, chunk_size=100, chunk_overlap=20, length_function=tiktoken_len):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.length_function = length_function
    
    def split_text(self, text):
        # First, split text by sentences
        sentences = text.split('. ')
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            # Check if adding the next sentence would exceed the chunk size
            if self.length_function(current_chunk + sentence) > self.chunk_size:
                if current_chunk:
                    chunks.append(current_chunk)
                current_chunk = sentence
            else:
                current_chunk += (". " if current_chunk else "") + sentence

        # Add any remaining text as the last chunk
        if current_chunk:
            chunks.append(current_chunk)
        
        # Apply overlap if necessary
        final_chunks = []
        for i in range(len(chunks)):
            start = max(i - self.chunk_overlap, 0)
            end = min(i + self.chunk_overlap + 1, len(chunks))
            combined_chunk = " ".join(chunks[start:end])
            final_chunks.append(combined_chunk)
        
        return final_chunks

# Usage
text_splitter = EnhancedTextSplitter(
    chunk_size=100,
    chunk_overlap=1,
    length_function=tiktoken_len,
)


In [76]:
chunks = text_splitter.split_text(CONTEXT)

In [48]:
len(chunks)

3

In [49]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at ev

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

####🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

1. `Dynamic chunk sizes to further preserve context.`
2. `Incresed overlap ebwteen chunks to maintain context`
3. `Split chunks at natural boundaries (end of sentences, end of paragraphs)`

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

If you come from an NLP background, embeddings are something you might be intimately familiar with - otherwise, you might find the topic a bit...dense. (this attempt at a joke will make more sense later)

In all seriousness, embeddings are a powerful piece of the NLP puzzle, so let's dive in!

> NOTE: While this notebook language/NLP-centric, embeddings have uses beyond just text!

### Why Do We Even Need Embeddings?

In order to fully understand what Embeddings are, we first need to understand why we have them!

Machine Learning algorithms, ranging from the very big to the very small, all have one thing in common:

They need numeric inputs.

So we need a process by which to translate the domain we live in, dominated by images, audio, language, and more, into the domain of the machine: Numbers.

Another thing we want to be able to do is capture "semantic information" about words/phrases so that we can use algorithmic approaches to determine if words are closely related or not!

So, we need to come up with a process that does these two things well:

- Convert non-numeric data into numeric-data
- Capture potential semantic relationships between individual pieces of data

### How Do Embeddings Capture Semantic Relationships?

In a simplified sense, embeddings map a word or phrase into n-dimensional space with a dense continuous vector, where each dimension in the vector represents some "latent feature" of the data.

This is best represented in a classic example:

![image](https://i.imgur.com/K5eQtmH.png)

As can be seen in the extremely simplified example: The X_1 axis represents age, and the X_2 axis represents hair.

The relationship of "puppy -> dog" reflects the same relationship as "baby -> adult", but dogs are (typically) hairier than humans. However, adults typically have more hair than babies - so they are shifted slightly closer to dogs on the X_2 axis!

Now, this is a simplified and contrived example - but it is *essentially* the mechanism by which embeddings capture semantic information.

In reality, the dimensions don't sincerely represent hard-concepts like "age" or "hair", but it's useful as a way to think about how the semantic relationships are captured.

Alright, with some history behind us - let's examine how these might help us choose relevant context.

Let's begin with a simple example - simply looking at how close to embedding vectors are for a given phrase.

When we use the term "close" in this notebook - we're referring to a distance measure called "cosine similarity".

We discussed above that if two embeddings are close - they are semantically similar, cosine similarity gives us a quick way to measure how similar two vectors are!

Closeness is measured from 1 to -1, with 1 being extremely close and -1 being extremely close to opposite in meaning.

Let's implement it with Numpy below.

In [50]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

We're going to be using OpenAI's `text-embedding-3-small` today.

In order to choose our embeddings model, we'll refer to the MTEB leaberboard - which can be found [here](https://huggingface.co/spaces/mteb/leaderboard)!

The basic logic is: We sort by our desired task - in this case `Retrieval Average (15 Datasets)`, and we're going to pick a model that performs well on that task - to keep cost in mind, we'll go with the `text-embedding-3-small` over the `text-embedding-3-large` since there's only a separation of ~5 points between the two on this task - but the cost is a significant factor less for the `small` version of the model.

In [51]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

Let's grab some vectors and see how they're related!

In [60]:
puppy_vec = embedding_model.embed_query("pup")
dog_vec = embedding_model.embed_query("dog")

Let's do a quick check to ensure they're all the correct dimension.

####❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

1536

Now, let's see how "puppy" and "dog" are related to eachother!

In [61]:
cosine_similarity(puppy_vec, dog_vec)

0.5044208833844728

We can repeat the experiment for things we might expect to be unrelated, as well:



In [63]:
puppy_vec = embedding_model.embed_query("puppy")
ice_vec = embedding_model.embed_query("ice cube")

In [64]:
cosine_similarity(puppy_vec, ice_vec)

0.20365601127332955

As expected, we get an unrelated score!

Great!

Now, let's extend it to our example.

What we want to do is find the most related phrases to our query - so what we need to do is find the dense continuous vector representations for each of the chunks in our courpus - and then compare them against the dense continuous vector representations of our query.

In simpler terms:

Compare the embedding of our query with the embeddings of each of our chunks!

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [77]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [79]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 1536



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
---
Embedding - Vector of Size: 1536



Chunk - Seamless LangSmith Tracing Integration As your chains get more and

Okay, great. Let's create a query - and then embed it!

In [80]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [81]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.5372562043448947


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [82]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [83]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [84]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?',
 'response': AIMessage(content='Yes, LCEL can help take code from the notebook to production by providing full sync, async, batch, and streaming support for chains constructed in this way. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface for production use.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 152, 'total_tokens': 215}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d6a25ce-9d48-4c11-9371-36ddb18e9cd4-0'),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, asyn

####❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use your newly created `simple_rag` to help you answer this question!

In [85]:
simple_rag("What does LCEL do that makes it more reliable at scale?", embeddings_dict, embedding_model, chat_chain)

{'query': 'What does LCEL do that makes it more reliable at scale?',
 'response': AIMessage(content='LCEL provides full sync, async, batch, and streaming support automatically for any chain constructed using this language. This makes it more reliable at scale as it can easily handle different types of data processing and execution scenarios.', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 153, 'total_tokens': 196}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea489254-4d9f-4fc7-9855-fe73c0f976a5-0'),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain i

# 🤝 Breakout Room #2

## Task #3: Create a Simple RAG Application Using Qdrant, OpenAI, and LCEL

Now that we have a grasp on how LCEL works, and how we can use LangChain and OpenAI to interact with our data - let's step it up a notch and incorporate Qdrant!

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using Drant as our `VectorStore` today. You can read more about it [here](https://qdrant.tech/documentation/).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

Let's use [The Ultimate Hitchhiker's Guide](https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf) as our data today!

### Data Collection

We'll be leveraging the `PyMUPDFLoader` to load our PDF directly from the web!

In [86]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://www.deyeshigh.co.uk/downloads/literacy/world_book_day/the_hitchhiker_s_guide_to_the_galaxy.pdf").load()

### Chunking Our Documents

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

In [87]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [88]:
len(split_chunks)

517

Alright, now we have 516 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [89]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

189


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `text-embedding-3-small` embedding model again - and `Qdrant` to store all our embedding vectors for easy retrieval later!

In [90]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="Hitchiker's Guide",
)

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [91]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

####🏗️ Activity #2:

Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.

In [92]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

<< COMPLETE YOUR PROMPT HERE >>
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

#### Our RAG Chain

Notice how we have a bit of a more complex chain this time - that's because we want to return our sources with the response.

Let's break down the chain step-by-step:

1. We invoke the chain with the `question` item. Notice how we only need to provide `question` since both the retreiver and the `"question"` object depend on it.
  - We also chain our `"question"` into our `retriever`! This is what ultimately collects the context through Qdrant.
2. We assign our collected context to a `RunnablePassthrough()` from the previous object. This is going to let us simply pass it through to the next step, but still allow us to run that section of the chain.
3. We finally collect our response by chaining our prompt, which expects both a `"question"` and `"context"`, into our `llm`. We also, collect the `"context"` again so we can output it in the final response object.

The key thing to keep in mind here is that we need to pass our context through *after* we've retrieved it - to populate the object in a way that doesn't require us to call it or try and use it for something else.

In [93]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

Let's get a visual understanding of our chain!

In [94]:
%pip install -qU grandalf

Note: you may need to restart the kernel to use updated packages.


In [95]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

Let's try another visual representation:

![image](https://i.imgur.com/Ad31AhL.png)

Let's test our chain out!

In [96]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the significance of towels in Douglas Adam's Hitchhicker's Guide?"})

In [97]:
response["response"].content

"In Douglas Adam's Hitchhiker's Guide, towels are considered to be incredibly useful and important items. The guide mentions that a towel is the most massively useful thing a hitchhiker can have. It can be used for various purposes such as waving as a distress signal, drying off, and even has immense psychological value. The presence of a towel is also seen as a sign of a capable and resourceful individual, as someone who knows where their towel is can be relied upon in challenging situations. The concept of towels in the Hitchhiker's Guide emphasizes their practicality and symbolic importance in the world of interstellar travel."

In [98]:
for context in response["context"]:
  print("Context:")
  print(context)
  print("----")

Context:
page_content="28  /  D O U G L A S  A D A M S  \nthis device was in fact that most remarkable of all books ever to \ncome out of the great publishing corporations of Ursa Minor - \nThe Hitch Hiker's Guide to the Galaxy.  The reason why it was \npublished in the form of a micro sub meson electronic \ncomponent is that if it were printed in normal book form, an \ninterstellar hitch hiker would require several inconveniently \nlarge buildings to carry it around in. \nBeneath that in Ford Prefect's satchel were a few biros, a \nnotepad, and a largish bath towel from Marks and Spencer. \nThe Hitch Hiker's Guide to the Galaxy has a few things to say on \nthe subject of towels. \nA towel, it says, is about the most massively useful thing an" metadata={'source': 'https://www.deyeshigh.co.uk/downloads/literacy/world_book_day/the_hitchhiker_s_guide_to_the_galaxy.pdf', 'file_path': 'https://www.deyeshigh.co.uk/downloads/literacy/world_book_day/the_hitchhiker_s_guide_to_the_galaxy.pdf', '

Let's see if it can handle a query that is totally unrelated to the source documents.

In [101]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Who is James Marnie Argarin?"})

In [102]:
response["response"].content

'James Marnie Argarin is not mentioned in the provided context. It seems like they are not a character or entity related to the text from "Hitchhiker\'s Guide to the Galaxy" by Douglas Adams.'

####❓ Question #4:

Where does Arthur Dent meet Marvin?

> HINT: Use your RAG Chain to answer this question.


In [104]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Where does Arthur Dent meet Marvin?"})
response["response"].content

"Arthur Dent meets Marvin in the corridor where Marvin is complaining about a pain in all the diodes down his left-hand side. They are walking along together when Marvin stops abruptly and points out something that has happened, catching Arthur's attention."

In [105]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Who is Arthur Dent and Marvin?"})
response["response"].content

'Arthur Dent is a human character in "Hitchhiker\'s Guide to the Galaxy" by Douglas Adams. He is a friend of Ford Prefect, who is actually from a small planet near Betelgeuse. Arthur Dent is the main character who encounters various adventures throughout the story. Marvin is a robot character in the same story who often displays a negative and pessimistic attitude.'

In [106]:
response = retrieval_augmented_qa_chain.invoke({"question" : "In one paragraph, give a summary of the book."})
response["response"].content

'"The Hitchhiker\'s Guide to the Galaxy" is a science fiction novel by Douglas Adams that follows the adventures of Arthur Dent, who is rescued from Earth\'s destruction by his alien friend Ford Prefect. Together, they explore the galaxy and encounter a variety of bizarre and humorous situations. The guide itself is a pivotal element in the story, as it provides information on everything in the universe. The book humorously explores themes of absurdity, philosophy, and the unknown, making it a beloved classic in the science fiction genre.'